In [0]:
import torch
from torch import nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import pickle

from pathlib import Path

import matplotlib.pyplot as plt

import pandas as pd

from fastai.basic_train import Learner, Callback
from fastai.basic_data import DataBunch
from fastai.vision.data import ImageList
from fastai.vision.models import resnet50, resnet18
from fastai.vision.transform import get_transforms
from fastai.metrics import accuracy

import torchvision.transforms as transforms

import os
import glob
import shutil
import random

In [2]:
# Courtesy of https://forums.fast.ai/t/how-does-one-download-imagenet/40660/9
!wget http://files.fast.ai/data/imagenet-sample-train.tar.gz

--2020-04-18 23:57:32--  http://files.fast.ai/data/imagenet-sample-train.tar.gz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2232645974 (2.1G) [application/gzip]
Saving to: ‘imagenet-sample-train.tar.gz’

imagenet-sample-tra 100%[===================>]   2.08G  35.8MB/s    in 60s     

2020-04-18 23:58:32 (35.6 MB/s) - ‘imagenet-sample-train.tar.gz’ saved [2232645974/2232645974]



# Data

In [0]:
# From https://pytorch.org/hub/pytorch_vision_vgg/
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

## Content Data

In [3]:
!wget http://files.fast.ai/data/imagenet-csv.tgz

--2020-04-18 22:42:24--  http://files.fast.ai/data/imagenet-csv.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4549635 (4.3M) [application/x-gtar-compressed]
Saving to: ‘imagenet-csv.tgz’

imagenet-csv.tgz    100%[===================>]   4.34M  8.60MB/s    in 0.5s    

2020-04-18 22:42:25 (8.60 MB/s) - ‘imagenet-csv.tgz’ saved [4549635/4549635]



In [4]:
!tar -xvf imagenet-sample-train.tar.gz

Streaming output truncated to the last 5000 lines.
train/n04259630/n04259630_11233.JPEG
train/n02113186/
train/n02113186/n02113186_4618.JPEG
train/n02113186/n02113186_2010.JPEG
train/n02113186/n02113186_12627.JPEG
train/n02113186/n02113186_8336.JPEG
train/n02113186/n02113186_10431.JPEG
train/n02113186/n02113186_2861.JPEG
train/n02113186/n02113186_961.JPEG
train/n02113186/n02113186_7075.JPEG
train/n02113186/n02113186_11222.JPEG
train/n02113186/n02113186_13841.JPEG
train/n02113186/n02113186_4198.JPEG
train/n02113186/n02113186_704.JPEG
train/n02113186/n02113186_7634.JPEG
train/n02113186/n02113186_1969.JPEG
train/n02113186/n02113186_19460.JPEG
train/n02113186/n02113186_12553.JPEG
train/n02113186/n02113186_9934.JPEG
train/n02113186/n02113186_10447.JPEG
train/n02113186/n02113186_5248.JPEG
train/n02113186/n02113186_632.JPEG
train/n02113186/n02113186_3456.JPEG
train/n02113186/n02113186_4784.JPEG
train/n02113186/n02113186_14760.JPEG
train/n02113186/n02113186_5457.JPEG
train/n02113186/n02113186_

In [5]:
!tar -xvf imagenet-csv.tgz

csv/
csv/valid.csv
csv/train.csv


In [0]:
labels = pd.read_csv("csv/train.csv")

In [7]:
labels.head()

,x,y
0,train/n07754684/n07754684_1900.JPEG,n07754684
1,train/n07754684/n07754684_27421.JPEG,n07754684
2,train/n07754684/n07754684_20089.JPEG,n07754684
3,train/n07754684/n07754684_34374.JPEG,n07754684
4,train/n07754684/n07754684_19961.JPEG,n07754684


In [0]:
content_image_size = 128

In [0]:
content_data = (ImageList.from_folder("train").
        split_by_rand_pct(0.2, seed = 567).
        label_from_folder(). #lambda x: os.path.basename(os.path.dirname(x))
        transform(size=content_image_size).
        normalize(mean, std).
        databunch())

In [0]:
content_train_ds = content_data.train_ds
content_valid_ds = content_data.valid_ds

## Style Data

In [0]:
# Upload Kaggle token first.
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json

In [15]:
!kaggle datasets download -d thedownhill/art-images-drawings-painting-sculpture-engraving

 99% 578M/583M [00:09<00:00, 57.7MB/s]
100% 583M/583M [00:09<00:00, 64.9MB/s]


In [16]:
!unzip art-images-drawings-painting-sculpture-engraving.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/dataset_updated/training_set/sculpture/i - 969.jpeg  
  inflating: dataset/dataset_updated/training_set/sculpture/i - 97.jpeg  
  inflating: dataset/dataset_updated/training_set/sculpture/i - 970.jpeg  
  inflating: dataset/dataset_updated/training_set/sculpture/i - 971.jpeg  
  inflating: dataset/dataset_updated/training_set/sculpture/i - 972.jpeg  
  inflating: dataset/dataset_updated/training_set/sculpture/i - 973.jpeg  
  inflating: dataset/dataset_updated/training_set/sculpture/i - 974.jpeg  
  inflating: dataset/dataset_updated/training_set/sculpture/i - 976.jpeg  
  inflating: dataset/dataset_updated/training_set/sculpture/i - 977.jpeg  
  inflating: dataset/dataset_updated/training_set/sculpture/i - 978.jpeg  
  inflating: dataset/dataset_updated/training_set/sculpture/i - 979.jpeg  
  inflating: dataset/dataset_updated/training_set/sculpture/i - 98.jpeg  
  inflating: dataset/dataset_updated/training_set/s

In [0]:
style_image_size = 128

In [19]:
style_train_ds = (ImageList.from_folder("musemart/dataset_updated/training_set/painting").
        split_none().
        label_empty(). #lambda x: os.path.basename(os.path.dirname(x))
        transform(size=style_image_size).
        databunch()).train_ds

You can deactivate this warning by passing `no_check=True`.


/usr/local/lib/python3.6/dist-packages/fastai/basic_data.py:262: UserWarning: There seems to be something wrong with your dataset, for example, in the first batch can't access these elements in self.train_ds: 485,1625,1578
  warn(warn_msg)


fastai.vision.image.Image

In [72]:
style_valid_ds = (ImageList.from_folder("musemart/dataset_updated/validation_set/painting").
        split_none().
        label_empty(). #lambda x: os.path.basename(os.path.dirname(x))
        transform([], size=400).
        databunch()).normalize([100, -2]).train_ds

You can deactivate this warning by passing `no_check=True`.


/usr/local/lib/python3.6/dist-packages/fastai/basic_data.py:262: UserWarning: There seems to be something wrong with your dataset, for example, in the first batch can't access these elements in self.train_ds: 62,75
  warn(warn_msg)


In [73]:
style_valid_ds[0][0].data

tensor([[[0.1255, 0.1246, 0.1218,  ..., 0.1216, 0.1216, 0.1216],
         [0.1255, 0.1246, 0.1218,  ..., 0.1216, 0.1216, 0.1216],
         [0.1255, 0.1246, 0.1218,  ..., 0.1236, 0.1236, 0.1236],
         ...,
         [0.1333, 0.1333, 0.1333,  ..., 0.1294, 0.1294, 0.1294],
         [0.1333, 0.1333, 0.1333,  ..., 0.1294, 0.1294, 0.1294],
         [0.1333, 0.1333, 0.1333,  ..., 0.1294, 0.1294, 0.1294]],

        [[0.1216, 0.1207, 0.1178,  ..., 0.1176, 0.1176, 0.1176],
         [0.1216, 0.1207, 0.1178,  ..., 0.1176, 0.1176, 0.1176],
         [0.1216, 0.1207, 0.1178,  ..., 0.1197, 0.1197, 0.1197],
         ...,
         [0.1294, 0.1294, 0.1294,  ..., 0.1255, 0.1255, 0.1255],
         [0.1294, 0.1294, 0.1294,  ..., 0.1255, 0.1255, 0.1255],
         [0.1294, 0.1294, 0.1294,  ..., 0.1255, 0.1255, 0.1255]],

        [[0.1059, 0.1050, 0.1021,  ..., 0.1020, 0.1020, 0.1020],
         [0.1059, 0.1050, 0.1021,  ..., 0.1020, 0.1020, 0.1020],
         [0.1059, 0.1050, 0.1021,  ..., 0.1040, 0.1040, 0.

In [0]:
class StyleTransferDS(Dataset):
  def __init__(self, content_ds, style_ds):
    self.content = content_ds
    self.style = style_ds
  def __getitem__(self, index):
    content = self.content[index]
    style = self.style[random.randint(0, len(self.style) - 1)]

    return content, style

  def __len__(self):
    return len(self.content)  

In [14]:
train_ds = StyleTransferDS(content_train_ds, style_train_ds)
valid_ds = StyleTransferDS(content_valid_ds, style_valid_ds)

NameError: ignored

# Architecture

In [0]:
class NeuralStyleTransfer(nn.Module):
  def __init__(self):
    self.vgg = torch.hub.load('pytorch/vision:v0.5.0', 'vgg11', pretrained=True)